### Helmholtz equation

$$u(x) - \lambda \nabla^2 u(x) = f(x)$$

with $f(x)=(1+\lambda\pi^2/4)u(x)$ the exact solution is

$u(x) = cos(\pi/2 x)$

In [35]:
import numpy as np
import scipy as sp
from pypde.bases.chebyshev import *
import matplotlib.pyplot as plt
from scipy.linalg import solve_triangular

def fun(x,lam):
    return  (1.0+lam*np.pi**2/4)*np.cos(np.pi/2*x)

N = 6
lam = 1/np.pi**2
CD = ChebDirichlet(N)
x = CD.x
I  = CD.mass.toarray()
D2 = CD.stiff.toarray()
f = fun(x,lam)
fhat = CD.forward_fft(f)

def rhs(fhat):
    return I@fhat

def lhs():
    return I-lam*D2


A = lhs()
b = rhs(fhat)


### Brute Force

In [36]:
%%timeit

uhat = np.linalg.solve(A,b)
u = CD.backward_fft(uhat)


#plt.plot(x,f)
#plt.plot(x,u,"b")
#plt.plot(x,np.cos(np.pi/2*x),"r--")
#plt.show()

544 µs ± 7.67 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


### LU Decomposiion

In [63]:
def TwoDMA_Solve(d, u1, x, axis=0):
    ''' 
    d: N
        diagonal
    u1: N-2
        Diagonal with offset +2
    x: array ndim==1
        rhs
    '''
    assert x.ndim == 1, "Use optimized version for multidimensional solve"
    n = d.shape[0]
    x[n-1] = x[n-1]/d[n-1]
    x[n-2] = x[n-2]/d[n-2]
    for i in range(n - 3, -1, -1):
        x[i] = (x[i] - u1[i]*x[i+2])/d[i]

def TwoDMA_SolveU(d, u1, x, axis=0):
    ''' 
    d: N
        diagonal
    u1: N-2
        Diagonal with offset -2
    x: array ndim==1
        rhs
    '''
    assert x.ndim == 1, "Use optimized version for multidimensional solve"
    n = d.shape[0]
    x[0] = x[0]/d[0]
    x[1] = x[1]/d[1]
    for i in range(2,n):
        x[i] = (x[i] - u1[i-2]*x[i-2])/d[i]
        
P,L,U = sp.linalg.lu(A)
bb = b.copy()

print(L)
b = np.random.random(N)

x1 = np.linalg.solve(L,b)
print(x1)

d,u1 = np.diag(L), np.diag(L,-2)
bb = b.copy()
TwoDMA_SolveU(d,u1,bb)

print(bb)

import tdma
print(tdma.__doc__)
bb = b.copy()
tdma.twodma_solve(d,u1,bb)
print(bb)

[[ 1.          0.          0.          0.          0.          0.        ]
 [ 0.          1.          0.          0.          0.          0.        ]
 [-0.38137129  0.          1.          0.          0.          0.        ]
 [ 0.         -0.08846249  0.          1.          0.          0.        ]
 [ 0.          0.         -0.25104888  0.          1.          0.        ]
 [ 0.          0.          0.         -0.42499375  0.          1.        ]]
[0.93420577 0.16258416 1.22625597 0.16060308 0.35124858 0.59635901]
[0.93420577 0.16258416 1.22625597 0.16060308 0.35124858 0.59635901]
This module 'tdma' is auto-generated with f2py (version:2).
Functions:
  twodma_solve(d,u1,x,n=len(d))
.
[0.93420577 0.16258416 1.22625597 0.16060308 0.35124858 0.59635901]


In [66]:
import pypde.bases.solver.utda as utda
print(utda.__doc__)

This module 'utda' is auto-generated with f2py (version:2).
Functions:
  x = solve_triangular(r,b,n=shape(r,0))
.


In [5]:
%%timeit
            
# L is almost pentadiagonal 
d,u1 = np.diag(L), np.diag(L,-2)


TwoDMA_SolveU(d,u1,bb)
uhat = solve_triangular(U,bb)
u = CD.backward_fft(uhat)

#plt.plot(x,f)
#plt.plot(x,u,"b")
#plt.plot(x,np.cos(np.pi/2*x),"r--")
#plt.show()

959 µs ± 47.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [6]:
#plt.spy(P)
#plt.show()
#plt.spy(L)
#print(L)
#plt.show()
#plt.spy(U)
#plt.show()

In [40]:
def TwoDMA_SolveU(d, u1, x, axis=0):
    ''' 
    d: N
        diagonal
    u1: N-2
        Diagonal with offset -2
    x: array ndim==1
        rhs
    '''
    assert x.ndim == 1, "Use optimized version for multidimensional solve"
    n = d.shape[0]
    x[0] = x[0]/d[0]
    x[1] = x[1]/d[1]
    for i in range(2,n - 1):
        x[i] = (x[i] - u1[i-2]*x[i-2])/d[i]
        
N = 6
a1 = np.random.random(N)*0+1
a2 = -np.random.random(N-2)
A = np.diag(a1,k=0) + np.diag(a2,k=-2)
b = np.random.random(N)
print(A)
bb = b.copy()
a1,a2 = np.diag(A), np.diag(A,-2)
TwoDMA_SolveU(a1,a2,bb)
print(bb)
b2 = np.linalg.solve(A,b)
print(b2)
np.allclose(bb,b2)


[[ 1.          0.          0.          0.          0.          0.        ]
 [ 0.          1.          0.          0.          0.          0.        ]
 [-0.38137129  0.          1.          0.          0.          0.        ]
 [ 0.         -0.08846249  0.          1.          0.          0.        ]
 [ 0.          0.         -0.25104888  0.          1.          0.        ]
 [ 0.          0.          0.         -0.42499375  0.          1.        ]]
[0.52156248 0.57364034 1.07128987 0.27665162 0.45838383 0.58301218]
[0.52156248 0.57364034 1.07128987 0.27665162 0.45838383 0.70058739]


False

In [20]:
A@bb

array([ -0.72511192,   3.5455193 ,  -0.23743854,   0.2602639 ,
         0.9235988 , -59.85356413])

In [19]:
b

array([-0.72511192,  3.5455193 , -0.23743854,  0.2602639 ,  0.9235988 ,
       -0.12756077])